In [1]:
from pLaplace_1D import test_runner_1D

# all possible types of inputs (not all combinations allowed)
_ = ["numpy", "numba", "jax"]  # "val_grad"
_ = ["Laplace", "SFD", "dense", "sparse"]  # "hess"
_ = ["my_newton", "my_trust_region", "scipy_trust-exact", "scipy_Newton-CG_hess", "scipy_BFGS", "scipy_CG", "scipy_L-BFGS-B", "scipy_TNC"]  # "minimizer"
_ = ["zero", "laplace"]  # "initial_guess"

problem_setting = {"p": 3,   # p in pLaplace
                   "a": -1,  # left bound of computational domain
                   "b": 1,   # right bound of computational domain
                   "f": "default"}  # source term, constant 10

sizes = [10, 100, 1000]


# Conclusions first
- fastest value and grad evaluation have numba (rewrite of matlab code)
- fastest method without the use of user provided hessian is `L-BFGS-B` (scipy)
- method similar to the "best" in the matlab code is `Newton-CG` (scipy) with approximation of hessian using numrical differentiation SFD (rewrite of matlab code)
- fastest method overall is my implementation of `newton` with linesearch using golden section with Laplace as a constant approximation of the hessian 

In [2]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["scipy_L-BFGS-B", "scipy_Newton-CG_hessp"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df, _, _, _ = test_runner_1D.create_tables(results, display_table="Ipython")


Function value:


,10,100,1000
numba/SFD/scipy L-BFGS-B/zero,-16.7724671,-16.8644661,-16.8654704
numba/SFD/scipy Newton-CG hessp/zero,-16.7724671,-16.8644661,-16.8654704
numba/Laplace/my newton/zero,-16.7724671,-16.8644660,-16.8654704


Time and iters:


$\mathcal{N}_{dof}$                          $n=10$              $n=100$  \
Option                                time prep/sol iters  time prep/sol   
numba/SFD/scipy L-BFGS-B/zero         0.718 / 0.000    16  0.224 / 0.004   
numba/SFD/scipy Newton-CG hessp/zero  0.718 / 0.002    12  0.224 / 0.009   
numba/Laplace/my newton/zero          0.320 / 0.001    15  0.234 / 0.002   

$\mathcal{N}_{dof}$                              $n=1000$        
Option                               iters  time prep/sol iters  
numba/SFD/scipy L-BFGS-B/zero          267  0.222 / 0.162  3280  
numba/SFD/scipy Newton-CG hessp/zero    19  0.222 / 0.209    19  
numba/Laplace/my newton/zero            37  0.239 / 0.011    63

# Solvers with only grad information
## Comparison of implementations (numpy vs numba vs jax)

In [3]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "Laplace",
                    "minimizer": ["scipy_TNC"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["scipy_TNC"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["scipy_TNC"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df, _, _, _ = test_runner_1D.create_tables(results, display_table="Ipython")


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Function value:


,10,100,1000
numpy/Laplace/scipy TNC/zero,-16.7724671,-16.8644661,-16.8654704
numba/Laplace/scipy TNC/zero,-16.7724671,-16.8644661,-16.8654704
jax/Laplace/scipy TNC/zero,-16.7724671,-16.8644661,-16.8654704


Time and iters:


$\mathcal{N}_{dof}$                  $n=10$              $n=100$        \
Option                        time prep/sol iters  time prep/sol iters   
numpy/Laplace/scipy TNC/zero  0.001 / 0.002    13  0.000 / 0.018    55   
numba/Laplace/scipy TNC/zero  0.262 / 0.001    13  0.330 / 0.006    57   
jax/Laplace/scipy TNC/zero    0.052 / 0.002    11  0.058 / 0.016    53   

$\mathcal{N}_{dof}$                $n=1000$        
Option                        time prep/sol iters  
numpy/Laplace/scipy TNC/zero  0.001 / 0.354   498  
numba/Laplace/scipy TNC/zero  0.231 / 0.224   558  
jax/Laplace/scipy TNC/zero    0.059 / 0.440   525

## For numba a comparison of multiple minimizators

In [4]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["scipy_TNC", "scipy_BFGS", "scipy_CG", "scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df, _, _, _ = test_runner_1D.create_tables(results, display_table="Ipython")


Function value:


,10,100,1000
numba/Laplace/scipy TNC/zero,-16.7724671,-16.8644661,-16.8654704
numba/Laplace/scipy BFGS/zero,-16.7724671,-16.8644661,-16.8654704
numba/Laplace/scipy CG/zero,-16.7724671,-16.8644661,-16.8654704
numba/Laplace/scipy L-BFGS-B/zero,-16.7724671,-16.8644661,-16.8654704


Time and iters:


$\mathcal{N}_{dof}$                       $n=10$              $n=100$        \
Option                             time prep/sol iters  time prep/sol iters   
numba/Laplace/scipy TNC/zero       0.255 / 0.001    13  0.225 / 0.006    57   
numba/Laplace/scipy BFGS/zero      0.255 / 0.001    21  0.225 / 0.020   121   
numba/Laplace/scipy CG/zero        0.255 / 0.001    40  0.225 / 0.026  1030   
numba/Laplace/scipy L-BFGS-B/zero  0.255 / 0.000    16  0.225 / 0.004   267   

$\mathcal{N}_{dof}$                      $n=1000$         
Option                              time prep/sol  iters  
numba/Laplace/scipy TNC/zero        0.343 / 0.223    558  
numba/Laplace/scipy BFGS/zero      0.343 / 18.777   1000  
numba/Laplace/scipy CG/zero         0.343 / 0.877  22977  
numba/Laplace/scipy L-BFGS-B/zero   0.343 / 0.163   3280

# Solvers using hessians 
**in scipy**
- `trust-exact` needs dense matrix, no reasonable warkaround
- `newton-cg` needs hessian-vector product

**own implementation of trust region method**

## Comparison of multiple implementation of Hessian approximation
- exact from jax autodiff - both dense and sparse version
- approx using SDF (copy of matlab code)
- constant approximation using Laplace (p=2)

In [5]:
all_solvers = []
all_solvers.append({"val_grad": "jax",
                    "hess": "dense",
                    "minimizer": ["scipy_trust-exact", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "sparse",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_trust_region", "scipy_Newton-CG_hessp"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df, _, _, _ = test_runner_1D.create_tables(results, display_table="Ipython")


Function value:


,10,100,1000
jax/dense/scipy trust-exact/zero,-16.7724671,-16.8644661,-16.8654704
jax/dense/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
jax/sparse/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
jax/SFD/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
jax/SFD/scipy Newton-CG hessp/zero,-16.7724671,-16.8644661,-16.8654704
jax/Laplace/my trust region/zero,-16.7724671,-16.8644661,-16.8654704


Time and iters:


$\mathcal{N}_{dof}$                        $n=10$              $n=100$        \
Option                              time prep/sol iters  time prep/sol iters   
jax/dense/scipy trust-exact/zero    0.133 / 0.003    10  0.132 / 0.033    54   
jax/dense/my trust region/zero      0.133 / 0.043     9  0.132 / 0.138    47   
jax/sparse/my trust region/zero     0.193 / 0.015     9  0.196 / 0.093    48   
jax/SFD/my trust region/zero        0.043 / 0.015     9  0.048 / 0.034    18   
jax/SFD/scipy Newton-CG hessp/zero  0.043 / 0.003    12  0.048 / 0.011    18   
jax/Laplace/my trust region/zero    0.041 / 0.020    14  0.047 / 0.092    43   

$\mathcal{N}_{dof}$                       $n=1000$        
Option                               time prep/sol iters  
jax/dense/scipy trust-exact/zero    0.148 / 13.162   500  
jax/dense/my trust region/zero       0.148 / 6.816   486  
jax/sparse/my trust region/zero      0.202 / 1.808   487  
jax/SFD/my trust region/zero         0.050 / 0.082    27  
jax/SFD/scipy Newton-CG hessp/zero   0.050 / 0.183    18  
jax/Laplace/my trust region/zero     0.048 / 0.199    73

# Comparison of implementations (numpy vs numba vs jax)
 - for trust region
 - also for newton method as both SFD and Laplace converge

In [6]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numpy",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df, _, _, _ = test_runner_1D.create_tables(results, display_table="Ipython")


Function value:


,10,100,1000
numpy/Laplace/my newton/zero,-16.7724671,-16.8644660,-16.8654704
numpy/Laplace/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
numba/Laplace/my newton/zero,-16.7724671,-16.8644660,-16.8654704
numba/Laplace/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
jax/Laplace/my newton/zero,-16.7724671,-16.8644660,-16.8654704
jax/Laplace/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
numpy/SFD/my newton/zero,-16.7724671,-16.8644661,-16.8654704
numpy/SFD/my trust region/zero,-16.7724671,-16.8644661,-16.8654704
numba/SFD/my newton/zero,-16.7724671,-16.8644661,-16.8654704
numba/SFD/my trust region/zero,-16.7724671,-16.8644661,-16.8654704


Time and iters:


$\mathcal{N}_{dof}$                        $n=10$              $n=100$        \
Option                              time prep/sol iters  time prep/sol iters   
numpy/Laplace/my newton/zero        0.001 / 0.003    15  0.000 / 0.011    37   
numpy/Laplace/my trust region/zero  0.001 / 0.020    14  0.000 / 0.072    43   
numba/Laplace/my newton/zero        0.267 / 0.001    15  0.265 / 0.003    37   
numba/Laplace/my trust region/zero  0.267 / 0.010    14  0.265 / 0.045    43   
jax/Laplace/my newton/zero          0.040 / 0.012    15  0.045 / 0.020    37   
jax/Laplace/my trust region/zero    0.040 / 0.020    14  0.045 / 0.092    43   
numpy/SFD/my newton/zero            0.000 / 0.002     8  0.001 / 0.003     8   
numpy/SFD/my trust region/zero      0.000 / 0.011     9  0.001 / 0.024    18   
numba/SFD/my newton/zero            0.233 / 0.002     8  0.225 / 0.002     8   
numba/SFD/my trust region/zero      0.233 / 0.006     9  0.225 / 0.015    18   
jax/SFD/my newton/zero              0.040 / 0.005     9  0.049 / 0.005     8   
jax/SFD/my trust region/zero        0.040 / 0.014     9  0.049 / 0.034    18   

$\mathcal{N}_{dof}$                      $n=1000$        
Option                              time prep/sol iters  
numpy/Laplace/my newton/zero        0.001 / 0.032    63  
numpy/Laplace/my trust region/zero  0.001 / 0.132    73  
numba/Laplace/my newton/zero        0.597 / 0.011    63  
numba/Laplace/my trust region/zero  0.597 / 0.090    73  
jax/Laplace/my newton/zero          0.048 / 0.064    63  
jax/Laplace/my trust region/zero    0.048 / 0.203    73  
numpy/SFD/my newton/zero            0.002 / 0.010    13  
numpy/SFD/my trust region/zero      0.002 / 0.061    25  
numba/SFD/my newton/zero            0.226 / 0.006    13  
numba/SFD/my trust region/zero      0.226 / 0.037    26  
jax/SFD/my newton/zero              0.048 / 0.016    13  
jax/SFD/my trust region/zero        0.048 / 0.084    27

# Initial guess as solution of Laplace (p=2)
Exact hessian (in jax autodiff) very much strougle when starting from 0. It is in orders of magnitude faster when starting from initial guess as solution of Laplace (p=2) (which is cheap in comparison).

In [7]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "sparse",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, compile_df, f_val_df, solve_df, _, _, _ = test_runner_1D.create_tables(results, display_table="Ipython")


Function value:


,10,100,1000
numba/Laplace/my newton/laplace,-16.7724671,-16.8644660,-16.8654704
jax/Laplace/my newton/laplace,-16.7724671,-16.8644660,-16.8654704
numba/SFD/my newton/laplace,-16.7724671,-16.8644661,-16.8654704
jax/SFD/my newton/laplace,-16.7724671,-16.8644661,-16.8654704
jax/sparse/my newton/laplace,-16.7724671,-16.8644661,-16.8654704


Time and iters:


$\mathcal{N}_{dof}$                     $n=10$              $n=100$        \
Option                           time prep/sol iters  time prep/sol iters   
numba/Laplace/my newton/laplace  0.275 / 0.000     8  0.661 / 0.002    35   
jax/Laplace/my newton/laplace    0.042 / 0.003     8  0.047 / 0.020    35   
numba/SFD/my newton/laplace      0.227 / 0.001     4  0.229 / 0.001     5   
jax/SFD/my newton/laplace        0.040 / 0.003     4  0.046 / 0.004     5   
jax/sparse/my newton/laplace     0.235 / 0.002     4  0.098 / 0.002     5   

$\mathcal{N}_{dof}$                   $n=1000$        
Option                           time prep/sol iters  
numba/Laplace/my newton/laplace  0.241 / 0.011    61  
jax/Laplace/my newton/laplace    0.047 / 0.062    61  
numba/SFD/my newton/laplace      0.226 / 0.004     8  
jax/SFD/my newton/laplace        0.048 / 0.010     8  
jax/sparse/my newton/laplace     0.101 / 0.005     6